In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras


print(tf.__version__)
print(sys.version_info)

for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.18.5
pandas 1.0.5
sklearn 0.21.2
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [3]:
# tf.function  and autograph
# tf.function可以使python语法变成tensorflow里的图
# autograph是tf.function所依赖的机制
# 概念比较一致

def scaled_elu(z,scale=1.0,alpha=1.0):
    # z > 0 ? scale * z : scale * alpha * tf .nn.elu(z) 
    is_positive = tf.greater_equal(z,0.0)
    return scale * tf.where(is_positive,z,alpha * tf.nn.elu(z))

print(scaled_elu(tf.constant(-3.)))
print(scaled_elu(tf.constant([-3.,-2.5])))

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)


In [8]:
# 变成tf中图实现的函数
scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.)))
print(scaled_elu_tf(tf.constant([-3.,-2.5])))   

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)


In [9]:
# 找回python函数
print(scaled_elu_tf.python_function is scaled_elu)

True


In [13]:
# 转换后运行速度会加快，tf内部有优化

%timeit scaled_elu(tf.random.normal((1000,1000)))
%timeit scaled_elu_tf(tf.random.normal((1000,1000)))

3.08 ms ± 22 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.8 ms ± 20.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
# 使用@tf.function装饰器,转换成tensorflow里的图

# 1 + 1/2 + 1/2^2 + ... + 1/2^n

@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /=  2.0 
    return total

# print(converge_to_2(20))

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [24]:
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display,Markdown
    display(Markdown("```python\n{}\n```".format(code)))
    
    
display_tf_code(scaled_elu)

ConversionError: converting <tensorflow.python.eager.def_function.Function object at 0x000001E8BE6E2BC8>: AttributeError: 'Function' object has no attribute '__code__'

In [25]:
display_tf_code(converge_to_2.python_function)

```python
def tf__converge_to_2(n_iters):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('converge_to_2', 'converge_to_2_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as converge_to_2_scope:
    total = ag__.converted_call(tf.constant, converge_to_2_scope.callopts, (0.0,), None, converge_to_2_scope)
    increment = ag__.converted_call(tf.constant, converge_to_2_scope.callopts, (1.0,), None, converge_to_2_scope)

    def get_state():
      return ()

    def set_state(_):
      pass

    def loop_body(iterates, increment, total):
      _ = iterates
      total += increment
      increment /= 2.0
      return increment, total
    increment, total = ag__.for_stmt(ag__.converted_call(range, converge_to_2_scope.callopts, (n_iters,), None, converge_to_2_scope), None, loop_body, get_state, set_state, (increment, total), ('increment', 'total'), ())
    do_return = True
    retval_ = converge_to_2_scope.mark_return_value(total)
  do_return,
  return ag__.retval(retval_)

```

In [31]:
var =  tf.Variable(0.)
@tf.function
def add_21():
    return var.assign_add(21) # +=

In [32]:
print(add_21())

tf.Tensor(21.0, shape=(), dtype=float32)


In [33]:
@tf.function
def cube(z):
    return tf.pow(z,3) # 立方

print(cube(tf.constant([1.,2.,3.])))
print(cube(tf.constant([1,2,3])))

tf.Tensor([ 1.  8. 27.], shape=(3,), dtype=float32)
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)
